# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

#### Import libraries

In [1]:
import timeit
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import joblib
import sys

from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.datasets import make_blobs
from sklearn.linear_model import LinearRegression, HuberRegressor, Lasso, Ridge
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector, SelectFromModel
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn import set_config

sys.modules['sklearn.externals.joblib'] = joblib
set_config(display="diagram")

#### Import the data into a pandas DataFrame

In [2]:
vehicles_raw = pd.read_csv('data/vehicles.csv')

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

#### We are asked to find the correlation bertween the key features of the data and what combination of feratures values, our model parameters, will influenc the price of a car.  
#### What is our customer profile? Are they technical? Are they professionals, recriational? Dothey chose the car based on 1, 2, 3 or more features?
#### I looked at the data from 2 perspective of the ttarget audience, 1 and the obvious is all the data for general use and the other is from a specific dealer ship perspective.

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

####  Our dataset contain 426880 samples of 18 features, including price, size, condition, make and model, year, odometer and more. These features could be used to calculate a price of a car. 
#### Most of the data features are categorical showing up as object data type.

In [3]:
vehicles_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

#### There is segnificant number of data missing, NaN. only 34868 samples include the full scope, that is only 8.1% of the dataset

In [5]:
vehicles_raw.isnull().sum()

id                   0
region               0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
VIN             161042
drive           130567
size            306361
type             92858
paint_color     130203
state                0
dtype: int64

In [7]:
vehicles_raw_copy = vehicles_raw

In [9]:
vehicles_raw_copy.dropna()

id      region  price    year manufacturer              model  \
126     7305672709      auburn      0  2018.0    chevrolet  express cargo van   
127     7305672266      auburn      0  2019.0    chevrolet  express cargo van   
128     7305672252      auburn      0  2018.0    chevrolet  express cargo van   
215     7316482063  birmingham   4000  2002.0       toyota               echo   
219     7316429417  birmingham   2500  1995.0          bmw               525i   
...            ...         ...    ...     ...          ...                ...   
426788  7303227528     wyoming  12995  2016.0    chevrolet           cruze lt   
426792  7303114896     wyoming  32999  2014.0         ford          f350, xlt   
426793  7303112347     wyoming  15999  2018.0    chevrolet          cruze, lt   
426833  7302338378     wyoming   6800  1997.0       jaguar    xk8 convertible   
426836  7302301268     wyoming      0  2018.0          ram               2500   

        condition    cylinders    fuel  odometer title_status transmission  \
126      like new  6 cylinders     gas   68472.0        clean    automatic   
127      like new  6 cylinders     gas   69125.0        clean    automatic   
128      like new  6 cylinders     gas   66555.0        clean    automatic   
215     excellent  4 cylinders     gas  155000.0        clean    automatic   
219          fair  6 cylinders     gas  110661.0        clean    automatic   
...           ...          ...     ...       ...          ...          ...   
426788   like new  4 cylinders     gas   61127.0        clean    automatic   
426792  excellent  8 cylinders  diesel  154642.0        clean    automatic   
426793  excellent  4 cylinders     gas   36465.0        clean    automatic   
426833       good  8 cylinders     gas   69550.0        clean    automatic   
426836  excellent  6 cylinders  diesel   20492.0        clean    automatic   

                      VIN drive       size         type paint_color state  
126     1GCWGAFP8J1309579   rwd  full-size          van       white    al  
127     1GCWGAFP4K1214373   rwd  full-size          van       white    al  
128     1GCWGAFPXJ1337903   rwd  full-size          van       white    al  
215     JTDBT123520243495   fwd    compact        sedan        blue    al  
219     WBAHD6322SGK86772   rwd   mid-size        sedan       white    al  
...                   ...   ...        ...          ...         ...   ...  
426788  1G1BE5SM0G7266569   fwd    compact        sedan      silver    wy  
426792  1FT8W3DT3EEB41548   4wd  full-size       pickup       brown    wy  
426793  1G1BE5SM7J7133704   fwd   mid-size        sedan       black    wy  
426833  SAJGX2749VCOO8376   rwd    compact  convertible       white    wy  
426836  3C6UR5HL3JG307217   4wd  full-size        truck       white    wy  

[34868 rows x 18 columns]

#### Just dropping the missing data is not good enough, we will have to selectively clean the data.

### Data Preparation

After our initial exploration and fine tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.